# https://docs.ray.io/en/latest/walkthrough.html

In [1]:
import ray
import time

In [2]:
# By adding the `@ray.remote` decorator, a regular Python function
# becomes a Ray remote function.
@ray.remote
def my_function():
    return 1

# To invoke this remote function, use the `remote` method.
# This will immediately return an object ref (a future) and then create
# a task that will be executed on a worker process.
obj_ref = my_function.remote()

# The result can be retrieved with ``ray.get``.
assert ray.get(obj_ref) == 1

In [3]:
@ray.remote
def slow_function():
    time.sleep(10)
    return 1

# Invocations of Ray remote functions happen in parallel.
# All computation is performed in the background, driven by Ray's internal event loop.
for i in range(4):
    # This doesn't block.
    ans = slow_function.remote()
    print(i, ray.get(ans))

0 1
1 1
2 1
3 1


In [4]:
@ray.remote
def function_with_an_argument(value):
    return value + 1


obj_ref1 = my_function.remote()
# assert ray.get(obj_ref1) == 1

# You can pass an object ref as an argument to another Ray remote function.
obj_ref2 = function_with_an_argument.remote(obj_ref1)
assert ray.get(obj_ref2) == 2

In [5]:
# Specify required resources.
@ray.remote(num_cpus=4)
def my_function():
    return 1
obj_ref1 = my_function.remote()
assert ray.get(obj_ref1) == 1

In [6]:
# Multiple Returns
@ray.remote(num_returns=3)
def return_multiple():
    return 1, 2, 3

a, b, c = return_multiple.remote()
print(ray.get([a, b, c]))

[1, 2, 3]


In [7]:
@ray.remote
def f(x):
    return x * x

futures = [f.remote(i) for i in range(4)]

print(ray.get(futures)) # [0, 1, 4, 9]

[0, 1, 4, 9]


In [8]:
@ray.remote(num_cpus=8)
def f(x):
    return x^2

futures = [f.remote(i) for i in range(10000)]

%timeit ray.get(futures) # [0, 1, 4, 9]

310 ms ± 4.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
@ray.remote(num_cpus=1)
def f(x):
    return x^2

futures = [f.remote(i) for i in range(10000)]

%timeit ray.get(futures) # [0, 1, 4, 9]

(f pid=22340) 


(f pid=12876) 
(f pid=22340) 


(f pid=1476) 


307 ms ± 7.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### NOTE: On checking CPU usage, it never went above 25% (8 core machine). Is parallelization working?


#### NOTE: There must be substantial work to do in parallel, else the speedup due to parallelization may not be justified by the overhead of parallelization.

Example: If we used range(100) above, there would not have been any speedup.

In [10]:
print(ray.is_initialized())
ray.shutdown()
print(ray.is_initialized())

True
False
